In [18]:
!pip install torchvision
!pip install matplotlib
!pip install pandas
!pip install torch torchvision
!pip install pillow
!pip install scikit-learn
!pip install tqdm



Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because norma

Import Libraries:

In [11]:
import pandas as pd
import json
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import os

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

Using: cuda


Data Loader:

In [13]:
# ✅ Unzip train2020.zip if not already extracted
import zipfile
import os

zip_path = 'train2020.zip'
extract_dir = 'train2020'

if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print(f"✅ Extracted dataset to: {extract_dir}")

# ✅ Imports
import json
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# ✅ Image transform (ResNet standard)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ✅ Step 1: Scan all .jpg files inside train2020/train/
image_paths = dict()
image_root = os.path.join('train2020', 'train')  # go into the 'train' subfolder

for root, _, files in os.walk(image_root):
    for file in files:
        if file.lower().endswith('.jpg'):
            file_id = os.path.splitext(file)[0].strip().lower()
            image_paths[file_id] = os.path.join(root, file)

print("📂 Total actual image files found:", len(image_paths))

# ✅ Step 2: Load and filter train.csv
df = pd.read_csv('train.csv')
df['ImageId'] = df['ImageId'].astype(str).str.strip().str.lower()

filtered_df = df[df['ImageId'].isin(image_paths)].copy()
filtered_df = filtered_df.dropna(subset=['ClassId'])
print("✅ Filtered dataset size:", len(filtered_df))

# ✅ Step 3: Define Dataset class
class FashionPediaCSVLoader(Dataset):
    def __init__(self, annotations_df, json_path, image_paths, transform=None):
        self.annotations = annotations_df.reset_index(drop=True)
        self.image_paths = image_paths
        self.transform = transform

        with open(json_path, 'r') as f:
            label_data = json.load(f)
        self.categories = label_data['categories']
        self.cat_id_to_name = {c['id']: c['name'] for c in self.categories}
        self.cat_id_to_idx = {cat_id: idx for idx, cat_id in enumerate(sorted(self.cat_id_to_name))}
        self.idx_to_name = {v: k for k, v in self.cat_id_to_idx.items()}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        for _ in range(10):
            row = self.annotations.iloc[idx]
            image_id = row['ImageId'].strip().lower()

            if image_id in self.image_paths:
                image_path = self.image_paths[image_id]
                try:
                    image = Image.open(image_path).convert("RGB")
                    if self.transform:
                        image = self.transform(image)
                    cat_id = int(row['ClassId'])
                    label = self.cat_id_to_idx[cat_id]
                    return image, label
                except Exception as e:
                    print(f"[Corrupt] Skipping: {image_path} — {e}")

            idx = (idx + 1) % len(self)

        raise FileNotFoundError("Too many missing or corrupt files in a row.")

# ✅ Step 4: Create Dataset + DataLoader
dataset = FashionPediaCSVLoader(
    annotations_df=filtered_df,
    json_path='label_descriptions.json',
    image_paths=image_paths,
    transform=transform
)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


✅ Extracted dataset to: train2020
📂 Total actual image files found: 45623
✅ Filtered dataset size: 333401


Load Dataset:

In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Split indices into train and val
indices = list(range(len(dataset)))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

train_subset = Subset(dataset, train_idx)
val_subset = Subset(dataset, val_idx)

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=2)


Pretrained ResNet-50 and Modify Final Layer

In [19]:
from torchvision import models
import torch.nn as nn # Import the 'nn' module
import torch.optim as optim # Import the 'optim' module from torch

model = models.resnet50(pretrained=True)

# Freeze all layers except the last FC
for param in model.parameters():
    param.requires_grad = False

# Replace final FC layer to match your number of classes
num_classes = len(dataset.cat_id_to_idx)
model.fc = nn.Linear(2048, num_classes)

model = model.to(device)


Loss Function:

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


Train Model:

In [21]:
from tqdm import tqdm

num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    print(f"\n📦 Epoch {epoch+1}/{num_epochs}")
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=True)

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update tqdm description
        loop.set_postfix({
            "Loss": f"{loss.item():.4f}",
            "Acc": f"{(100 * correct / total):.2f}%"
        })

    print(f"✅ Epoch {epoch+1} Done - Total Loss: {total_loss:.4f} - Accuracy: {(100 * correct / total):.2f}%")



📦 Epoch 1/2


Epoch 1: 100%|██████████| 8335/8335 [21:24<00:00,  6.49it/s, Loss=2.9847, Acc=19.61%]


✅ Epoch 1 Done - Total Loss: 23901.5794 - Accuracy: 19.61%

📦 Epoch 2/2


Epoch 2: 100%|██████████| 8335/8335 [21:53<00:00,  6.34it/s, Loss=2.6325, Acc=20.03%]

✅ Epoch 2 Done - Total Loss: 23485.1741 - Accuracy: 20.03%


Evaluate on Validation Set:

In [23]:
model.eval()
val_correct = 0
val_total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_acc = 100 * val_correct / val_total
print(f"Validation Accuracy: {val_acc:.2f}%")


KeyboardInterrupt: 